In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
from keras.models import load_model
from keras import backend as K
from create_gif import ReinforceAgent
import numpy as np

In [10]:
# Start virtual display
# pip install PyVirtualDisplay
# sudo apt-get install xvfb

from pyvirtualdisplay import Display
display = Display(visible=0, size=(1024, 768))
display.start()
import os
os.environ["DISPLAY"] = ":" + str(display.display) + "." + str(display.screen)

In [21]:
agent = ReinforceAgent('CartPole-v1')
# agent.model = load_model('logs/CartPole-v1/REINFORCE_CAUSAL/1_1_0.999_0.001_1574907638.hdf5', compile=False)
# agent.model = load_model('logs/CartPole-v1/REINFORCE_BASELINE/1_1_0.999_0.001_1574904904.hdf5', compile=False)
# agent.model = load_model('logs/CartPole-v1/REINFORCE/1_1_0.999_0.001_1574901446.hdf5', compile=False)
agent.model = load_model('logs/CartPole-v1/REINFORCE/5_1_0.999_0.001_1574902916.hdf5', compile=False)

In [22]:
_ = agent.get_eval_episode('REINFORCE_5_1_0.999_0.001_1574902916.gif')

![gif](REINFORCE_5_1_0.999_0.001_1574902916.gif)

![gif](REINFORCE_1_1_0.999_0.001_1574901446.gif)

![gif](REINFORCE_CAUSAL_1_1_0.999_0.001_1574907638.gif)

![gif](REINFORCE_BASELINE_1_1_0.999_0.001_1574904904.gif)